In [1]:
import pandas as pd
import numpy as np
from numpy import savetxt
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
import sys
# np.set_printoptions(threshold=sys.maxsize)
import os
import matplotlib.pyplot as plt
from sklearn import datasets
import us as usStates

In [2]:
states_in_the_prediction = ['01',
 '02',
 '04',
 '05',
 '06',
 '08',
 '09',
 '10',
 '12',
 '13',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '53',
 '54',
 '55',
 '56']

In [3]:
states_to_drop = ['11','US']
dfg = pd.read_csv('/Users/xiangyikong/Desktop/Flu_preprocess/truth-Incident Hospitalizations.csv')
dfg['date'] = pd.to_datetime(dfg['date'])
dfg_date_filtered = dfg[(dfg['date'] >= '2022-02-26') & (dfg['date'] < '2023-01-28')]
dfg_date_filtered = dfg_date_filtered[~dfg_date_filtered['location'].isin(states_to_drop)]
dfg_date_filtered_temp = dfg_date_filtered.sort_values(by=['date', 'location'])
dfg_date_filtered_temp = dfg_date_filtered[dfg_date_filtered['location'].isin(states_in_the_prediction)]
time_length = dfg_date_filtered_temp['date'].nunique()
state_length = dfg_date_filtered_temp['location'].nunique()
flu_groundtruth = np.resize(dfg_date_filtered_temp.groupby(['date','location']).sum().to_numpy().T, (time_length,state_length)) # 32,50
flu_groundtruth = flu_groundtruth.reshape(1,time_length,state_length)
print(flu_groundtruth.shape)
flu_groundtruth = flu_groundtruth.transpose(1,2,0)
print(flu_groundtruth.shape)

(1, 48, 50)
(48, 50, 1)


In [4]:
#x data

x_num_samples, x_num_nodes = flu_groundtruth.shape[0], flu_groundtruth.shape[1]
x_data = flu_groundtruth
# x_data = residual.transpose(1,2,0)

x_day = list(range(0,x_num_samples))

x_day = np.tile(x_day, [1, x_num_nodes, 1]).transpose((2, 1, 0))
x_data = np.concatenate([x_data,x_day], axis=-1)
print(x_data[:,:,1])

[[ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  1  1  1]
 [ 2  2  2 ...  2  2  2]
 ...
 [45 45 45 ... 45 45 45]
 [46 46 46 ... 46 46 46]
 [47 47 47 ... 47 47 47]]


In [5]:
x1 = np.stack(x_data, axis=0)[:time_length-1]
x1 = x1.reshape(time_length-1,1,state_length,2)

In [6]:
x1.shape

(47, 1, 50, 2)

In [7]:
y_data = flu_groundtruth[1:,:,:]
y1 = np.expand_dims(y_data,axis=1)
y1 = y1.transpose(0,3,2,1)
y_test = np.zeros([1,4,state_length,1])

In [ ]:
x_test = x1[-1].reshape(1,1,state_length,2)